In [182]:
from glob import glob
from os.path import splitext, basename
from xml.dom import minidom

In [183]:
def string2coord(string):
    '''take a string such as "x,y" and return a list of two floats [x,y]'''
    return [float(x) for x in string.split(',')]

def d_bezier_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing a relative bezier curves.
    Output is the list of points of this path (first point is absolute, others relative to previous ones), 
    without curve parameters.
    '''
    path_d = path_d.split(' ')
    points = list()
    points.append(string2coord(path_d[1]))
    
    for i in range(5, len(path_d), 3):
        points.append(string2coord(path_d[i]))      
    return points

def d_straigh_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing a relative straight path.
    Output is the list of points of this path (first point is absolute, others relative to previous ones)
    '''
    path_d = path_d.lstrip('m ').rstrip('z ').split(' ')
    points = list()
    for pt in path_d:
        points.append(string2coord(pt))
    return points

def d_qbezier_to_straight_path(path_d):
    '''
    Input is the d attribute of an svg path element representing an absolute quadratic beizer path.
    Typically produced by the pen tool of Concept (tested on Android).
    Output is the list of absolute points of this path.
    '''
    d_path2 = list()
    for i in allpath[0].getAttribute('d').split(' '):
        try:
            d_path2.append(float(i))
        except:
            pass
    
    points = list()
    for i in range(0, len(d_path2), 4):
        points.append([d_path2[i], d_path2[i+1]])    
    return points

def relative_path_to_absolute(points):    
    x = 0
    y = 0
    abs_path=list()
    for pt in points:
        x = pt[0] + x
        y = pt[1] + y
        abs_path.append([x,y])
    return abs_path

def translate_scale_round_path(viewBox,W_ratio, H_ratio, points):
    ts_points = list()
    for pt in points:
        x = int(round((pt[0] - viewBox[0])*W_ratio))
        y = int(round((pt[1] - viewBox[1])*H_ratio))
        ts_points.append([x,y])
    return ts_points


In [184]:
class svg2coco:
    def __init__(self, svgfolder, configfile):
        
        self.wd = svgfolder
        self.coco = {'images':[],
                     'annotations':[],
                     'categories':[],
                     'type':'',
                     'licenses': 'MyLicense',
                     'info': 'MyInfo'}

In [188]:
wd = './example'
svg_file = glob(wd+'/*.svg')

In [190]:
svg_file

['./example/test.svg', './example/test2.svg', './example/test3.svg']

In [196]:
img_id = 0
annot_id = 0
annotations = list()
images = list()
for svgf in svg_file:
    
    image_name = splitext(basename(svgf))[0] + '.jpg'
    doc = minidom.parse(svgf)
    # Collect svg viewbox
    viewBox = doc.getElementsByTagName('svg')[0].getAttribute('viewBox')
    viewBox = [float(n) for n in viewBox.split(' ')]
    viewBox_W, viewBox_H = viewBox[2] - viewBox[0], viewBox[3] - viewBox[1]
    W = doc.getElementsByTagName('svg')[0].getAttribute('width')
    W = int(float(W.replace('px', '')))
    H = doc.getElementsByTagName('svg')[0].getAttribute('height').replace('px', '')
    H = int(float(H.replace('px', '')))

    # Collect all paths
    allpath = [path for path in doc.getElementsByTagName('path')]

    W_ratio = W / viewBox_W
    H_ratio = H / viewBox_H
    
    # Prepare image informations
    images.append({'id': img_id,
                   'file_name':image_name,
                   'height':H,
                   'width':W})

    # Dealing with relative bezier curves paths
    for path in allpath:
        d = path.getAttribute('d')
        categorie = path.getAttribute('stroke')

        if 'c' in d:
            #print('Found c command, assuming relative Bezier curves')
            points = d_bezier_to_straight_path(d)
            points = relative_path_to_absolute(points)
            points = translate_scale_round_path(viewBox, W_ratio, H_ratio, points)
            
        elif 'Q' in d:
            #print('Found Q command, assuming Q Bezier curves')
            points = d_qbezier_to_straight_path(d)
            points = translate_scale_round_path(viewBox, W_ratio, H_ratio, points)
    
        elif 'm' in d:
            #print('assuming straight relative path')
            points = d_straigh_to_straight_path(d)
            points = relative_path_to_absolute(points)
            points = translate_scale_round_path(viewBox, W_ratio, H_ratio, points)
            
        # Getting minimal, unrotated,  bounding box coordinates
        minx = min([i[0] for i in points])
        maxx = max([i[0] for i in points])
        miny = min([i[1] for i in points])
        maxy = max([i[1] for i in points])
        
        points = [pt for sublist in points for pt in sublist]
            
        annotations.append({'id':annot_id,
                            'categorie_name': categorie,
                            'segmentation': points,
                            'image_id': img_id,
                            'iscrowd': 0,
                            'bbox:':[minx, miny, maxx-minx, maxy-miny],
                            'area': (maxx-minx) * (maxy-miny)})
        annot_id+=1
    img_id+=1

# Indexing categories:
categories = list()
cat = list(set([a['categorie_name'] for a in annotations]))
cat.sort()
mapper = dict(zip(cat,range(0,len(cat),1)))

# Making the COCO categories field. Supercategory remains blank (unused).
for k,v in mapper.items():
    categories.append({'id': v,
                    'name':k,
                    'supercategory':''})
    
# Adding category id to annotations.
for a in annotations:
    a['categorie_id'] = mapper[a['categorie_name']]
    del a['categorie_name']

In [197]:
coco = {'images':images,
        'annotations':annotations,
        'categories':categories,
        'type':'',
        'licenses': 'Clem',
        'info': 'Clem'}